In [35]:
import sys
sys.setrecursionlimit(1_000_000)

In [36]:
with open("day23.txt") as f:
    lines = f.readlines()
current_pattern = []
for i, line in enumerate(lines):
    if i != len(lines) -1:
            line = line[:-1]
    if len(line) > 3:
        line = line[:-1]
        current_pattern.append(line)
current_pattern

['#.##########################################################################################################################################',
 '#.#####...###...#####...#...###.......#.....#...#####...#...###...#.........###...#...#.......#...#...#...###.............###...###...###...',
 '#.#####.#.###.#.#####.#.#.#.###.#####.#.###.#.#.#####.#.#.#.###.#.#.#######.###.#.#.#.#.#####.#.#.#.#.#.#.###.###########.###.#.###.#.###.#.',
 '#...#...#...#.#.###...#...#...#.....#.#.#...#.#...#...#.#.#...#.#.#.......#...#.#.#.#.#.....#.#.#.#.#.#.#...#...........#.#...#.....#.#...#.',
 '###.#.#####.#.#v###.#########.#####.#.#.#.###.###.#.###.#.###.#.#.#######.###.#.#.#.#.#####.#.#.#.#.#.#.###.###########.#.#.#########.#.###.',
 '###.#...###.#.#.>.#.#.......#.....#.#.#.#.#...#...#...#...#...#.#.#.....#.#...#.#.#.#...###.#.#.#.#.#.#.#...#...#.....#.#...#.........#.#...',
 '###.###.###.#.#v#.#.#.#####.#####.#.#.#.#.#.###.#####.#####.###.#.#.###.#.#.###.#.#.###.###.#.#.#.#.#.#.#.###.#.#.###.#.#####.##

In [37]:
x_dim = len(current_pattern[0])
y_dim = len(current_pattern)
x_dim, y_dim

(140, 141)

In [38]:
import numpy as np
symbols = np.zeros((y_dim, x_dim), dtype="str")

In [39]:
numbers = []
for y, row in enumerate(current_pattern):
    for x, val in enumerate(row):
        symbols[y, x] = val


In [40]:
np.set_printoptions(edgeitems=30, linewidth=100000)
def pretty_print(matrix):
    print(str(matrix).replace(' ','').replace("'","").replace('[','').replace(']',''))

In [41]:
pretty_print(symbols)

#.############################...##############################
#.#####...###...#####...#...##...............###...###...###...
#.#####.#.###.#.#####.#.#.#.##...###########.###.#.###.#.###.#.
#...#...#...#.#.###...#...#................#.#...#.....#.#...#.
###.#.#####.#.#v###.#########....#########.#.#.#########.#.###.
###.#...###.#.#.>.#.#.......#......#.....#.#...#.........#.#...
###.###.###.#.#v#.#.#.#####.##...#.#.###.#.#####.#########.#.##
###.#...#...#.#.#...#.#...#.#....#.#...#.#.#...#...........#...
###.#.###.###.#.#####.#.#.#.#....#.###.#.#.#.#.###############.
###...###.....#...###.#.#.#......#...#.#.#...#.#.........#...#.
#################.###.#.#.####...###.#.#.#####.#.#######.#.#.#.
#...#...#...#...#...#.#.#.#.>....#...#.#.#...#.#.......#.#.#.#.
#.#.#.#.#.#.#.#.###.#.#.#.#.#v...#.###.#.#.#.#.#######.#.#.#.#.
#.#...#...#...#.....#...#...#....#.....#...#...#.......#...#...
#.############################...###############.##############
#.#.........#...#.........###...........

In [42]:
def can_visit(destination):
    y, x = destination
    y_max = symbols.shape[0]
    x_max = symbols.shape[1]
    if x < 0 or y < 0:
        return False
    if x >= x_max or y >= y_max:
        return False
    symbol = symbols[y, x]
    if  symbol == '#':
        return False    
    return  True

In [43]:
junctions = set()
starting_node = None
terminating_node = None
for y, row in enumerate(current_pattern):
    for x, val in enumerate(row):
      if symbols[y, x] != '#':
            possible_directions = 0
            if can_visit((y-1, x)):
                  possible_directions+=1
            if can_visit((y+1, x)):
                  possible_directions+=1
            if can_visit((y, x+1)):
                  possible_directions+=1
            if can_visit((y, x-1)):
                  possible_directions+=1
            if possible_directions >2 or y == 0 or y==y_dim-1:
                  junctions.add((y, x))
                  if y == 0:
                       starting_node = (y, x)
                  if y == y_dim -1:
                       terminating_node = (y, x)
len(junctions)

36

In [44]:
starting_node, terminating_node

((0, 1), (140, 139))

In [45]:
neighbors = {}
distances = {}

In [46]:
def find_next_junction(current_node, visited_nodes):
    if current_node in junctions:
        return current_node, len(visited_nodes)-1
    y, x = current_node
    up = (y-1, x)
    down = (y+1, x)
    left = (y, x-1)
    right = (y, x+1)
    for direction in [up, down, left, right]:
        if can_visit(direction) and direction not in visited_nodes:
            visited_nodes_copy = visited_nodes.copy()
            visited_nodes_copy.add(direction)
            return find_next_junction(direction, visited_nodes_copy)
    

In [47]:
for junction in junctions:
    neighbors[junction] = []
for junction in junctions:
    y, x = junction
    up = (y-1, x)
    down = (y+1, x)
    left = (y, x-1)
    right = (y, x+1)
    for direction in [up, down, left, right]:
        if can_visit(direction):
            neighbor, distance = find_next_junction(direction, {junction, direction})
            neighbors[junction].append(neighbor)
            distances[y,x,neighbor[0], neighbor[1]] = distance
    

In [48]:
neighbors

{(17, 113): [(33, 109), (13, 79), (35, 123)],
 (37, 55): [(13, 55), (59, 63), (29, 39), (31, 79)],
 (55, 127): [(35, 123), (79, 129), (59, 111)],
 (63, 13): [(43, 19), (81, 9), (63, 41)],
 (33, 109): [(17, 113), (59, 111), (31, 79), (35, 123)],
 (111, 77): [(87, 83), (135, 89), (99, 53), (99, 109)],
 (87, 83): [(53, 85), (111, 77), (85, 67), (81, 113)],
 (99, 109): [(81, 113), (137, 113), (111, 77), (101, 131)],
 (85, 67): [(59, 63), (99, 53), (75, 43), (87, 83)],
 (135, 89): [(111, 77), (133, 67), (137, 113)],
 (11, 29): [(29, 39), (5, 15), (13, 55)],
 (133, 67): [(99, 53), (129, 41), (135, 89)],
 (59, 63): [(37, 55), (85, 67), (63, 41), (53, 85)],
 (113, 31): [(75, 43), (129, 41), (111, 11), (99, 53)],
 (99, 53): [(85, 67), (133, 67), (113, 31), (111, 77)],
 (0, 1): [(5, 15)],
 (5, 15): [(0, 1), (43, 19), (11, 29)],
 (137, 113): [(99, 109), (135, 89), (129, 135)],
 (35, 123): [(17, 113), (55, 127), (33, 109)],
 (79, 129): [(55, 127), (101, 131), (81, 113)],
 (140, 139): [(129, 135)],

In [49]:
distances

{(17, 113, 33, 109): 60,
 (17, 113, 13, 79): 350,
 (17, 113, 35, 123): 420,
 (37, 55, 13, 55): 96,
 (37, 55, 59, 63): 162,
 (37, 55, 29, 39): 88,
 (37, 55, 31, 79): 142,
 (55, 127, 35, 123): 236,
 (55, 127, 79, 129): 206,
 (55, 127, 59, 111): 52,
 (63, 13, 43, 19): 170,
 (63, 13, 81, 9): 82,
 (63, 13, 63, 41): 140,
 (33, 109, 17, 113): 60,
 (33, 109, 59, 111): 124,
 (33, 109, 31, 79): 184,
 (33, 109, 35, 123): 52,
 (111, 77, 87, 83): 102,
 (111, 77, 135, 89): 220,
 (111, 77, 99, 53): 132,
 (111, 77, 99, 109): 224,
 (87, 83, 53, 85): 216,
 (87, 83, 111, 77): 102,
 (87, 83, 85, 67): 58,
 (87, 83, 81, 113): 188,
 (99, 109, 81, 113): 74,
 (99, 109, 137, 113): 274,
 (99, 109, 111, 77): 224,
 (99, 109, 101, 131): 148,
 (85, 67, 59, 63): 166,
 (85, 67, 99, 53): 68,
 (85, 67, 75, 43): 122,
 (85, 67, 87, 83): 58,
 (135, 89, 111, 77): 220,
 (135, 89, 133, 67): 124,
 (135, 89, 137, 113): 98,
 (11, 29, 29, 39): 92,
 (11, 29, 5, 15): 84,
 (11, 29, 13, 55): 168,
 (133, 67, 99, 53): 240,
 (133, 67, 1

In [50]:
def get_longest_path(current_junction, visited_junctions, total_path):
    y, x = current_junction
    if y == y_dim - 1:
        return total_path
    path_lengths = []
    for neighbor in neighbors[current_junction]:
        if neighbor not in visited_junctions:
            visited_nodes_copy = visited_junctions.copy()
            visited_nodes_copy.add(neighbor)
            additional_distance= distances[y, x, neighbor[0], neighbor[1]]
            path_lengths.append(get_longest_path(neighbor, visited_nodes_copy, total_path+additional_distance))
    if len(path_lengths) == 0:
        return -1
    return max(path_lengths)
        



In [51]:
s = {starting_node}
get_longest_path(starting_node, s, 0)

6710